# Neural Matching Models for Question Retrieval and Next Question Prediction in Conversation
## RNN CNN Match
### Implementation
#### https://arxiv.org/pdf/1707.05409.pdf

# Задачи

1. Подготовка данных (отчистка и векторизация). Разбиение на train/test
1. Реализация модели LSTM-CNN-Match. В качестве функции потерь использовать бинарную кросс-энтропию. (Помните, что у двух башен модели должны быть одинаковые веса)
1. Обучение.
1. Выводы по метрикам/лоссу. Общие выводы по модели
1. Сделать коммит на kaggle.

# Решение

Jupyter Notebooks хорошее решение, но иногда не самое практичное и порой далеко не самое воспроизводимое. Я хотел сделать небольшую архитектуру проекта для этого задания, чтобы можно было легко менять нужные мне блоки сети, чтобы код и подход был понятней, быстрее эксперементировать, менять нужные мне части, например составление датасета, генерацию негативных примеров.

## Архитектура проекта
![alt text](./images/my_arch.png)

## Dataset
**Цель** — перевести файл с данными в массив из индексов.  
**Описание** — читаем файлы, чистим данные, создаем словарь, подготавливаем батч. Читает и `test.csv` и `sample_submission.csv`, потому что тестовый файл кривой и его надо преобразовать, то есть для воспроизводимости.
## Cleaner
**Цель** — очистить текст от ненужной информации, например удаление ненужных символов, замена чисел и тд.  
**Описание** — На этом пукте можно сидеть вечно, по разному преобразовывая текст, к тому же очень много кернелов на кейгле по предобработке текста конкретно по этой задаче. Поэтому я не ставил себе цель много времени уделять этому пукту, оставив для этого возможность, если будет достаточно свободного времени.
## Layers
**Цель** — сделать основные слои.  
**Описание** — такой подход нужен для переиспользованиия и контроля этих слоев. Например, в слое CNN в методе forward есть транспонирование, в RNN есть упаковка последовательностей разной длины при заданном `x_lengths`. 
## Models
**Цель** — контроль моделей.  
**Описание** — есть возможность изменять обособленно только архитектуру модели.
## Templates
**Цель** — основные методы для работы с моделью.     
**Описание** — если в `Models` задается только архитектура башни, то здесь вся модель и необходимые для нее методы, например `text_embedding` для эмбеддинга батча текстов без расчета градиентов, расчет реколла, лосса и тд.
## Wrapper
**Цель** — основной класс для работы с моделью и данными.    
**Описание** — здесь можно задать то, как будет учиться модель и на каких данных, учить модель, сохранить лучшую, вывести метрики, построить графики, например, лосса, сделать сабмит.
### Hard negative generation
1. Берем query батч
1. Берем рандомные примеры, количество задается `batch_size * hard_negatives_multiplier`, но не больше `max_hard_negatives`, чтобы влезть в память.
1. Векторизуем query батч и рандомные примеры из предыдущего пункта
1. Матрично перемножаем (рандомные примеры транспонируются), получаем матрицу размером `(batch_size, batch_size * hard_negatives_multiplier)`
1. Из нее с помощью `argmax` достаем наиболее релевантные примеры. Нам не важно насколько похожи примеры, нам важно, чтобы они были максимально похожи среди тех, что мы выбрали
1. Если задан параметр `hard_k_next`, то из получившийся матрицы вычитаем матрицу такого же размера, где все нули, кроме `argmax` индексов, где сохраняются значения. Мотивация: не выбирать самый соответствующий пример, а выбирать следующий. Например, чтобы избежать выбор самого элемента при большом размере `batch_size` и `batch_size * hard_negatives_multiplier`
1. Отдаем релевантные примеры

# Пример

In [1]:
# импорты
import torch
from tools import config, Wrapper, DatasetQuora
from modelling.models import RNNCNNMatch
from modelling.templates import SimilarityTemplate

In [2]:
# задаем класс датасета
dataset = DatasetQuora(train_file=config.TRAIN_FILE,
                       test_file=config.TEST_FILE, 
                       sample_submission_file=config.SAMPLE_SUBMISSION_FILE)

In [3]:
# задаем модель
rnn_cnn_match = SimilarityTemplate(
    query_model=RNNCNNMatch(), 
    vocab_size=len(dataset.token2index),
    loss_type='cross_entropy')

In [4]:
# задаем оптимизатор
optimizer = torch.optim.Adam(rnn_cnn_match.parameters())

In [5]:
# задаем обертку
rcm = Wrapper(dataset=dataset, 
              model=rnn_cnn_match, 
              optimizer=optimizer, 
              model_name=config.MODEL_NAME, 
              batch_size=32,
              generate_negatives_type='hard', 
              hard_negatives_multiplier=32, 
              validation_batch_size_multiplier=32)

In [6]:
# обучение, по дефолту - 5 эпох
# rcm.train(verbose=config.VERBOSE, save_best=True)

# Архитектура сети

![alt text](./images/arch.png)

## PyTorch

In [8]:
rnn_cnn_match

SimilarityTemplate(
  (query_embedding_layer): Embedding(79279, 300, padding_idx=0)
  (candidate_embedding_layer): Embedding(79279, 300, padding_idx=0)
  (query_model): RNNCNNMatch(
    (fully_connected): Linear(in_features=896, out_features=300, bias=True)
    (model): Sequential(
      (0): RNN(
        (rnn): LSTM(300, 256)
      )
      (1): CNN(
        (convolution_layer): Conv1d(256, 128, kernel_size=(1,), stride=(1,))
        (activation_function): GELU()
        (pool_layer): MaxPool1d(kernel_size=4, stride=1, padding=0, dilation=1, ceil_mode=False)
      )
      (2): CNN(
        (convolution_layer): Conv1d(128, 128, kernel_size=(2,), stride=(1,))
        (activation_function): GELU()
        (pool_layer): MaxPool1d(kernel_size=4, stride=1, padding=0, dilation=1, ceil_mode=False)
      )
      (3): CNN(
        (convolution_layer): Conv1d(128, 128, kernel_size=(3,), stride=(1,))
        (activation_function): GELU()
        (pool_layer): MaxPool1d(kernel_size=4, stride=1, pad

# RNN CNN Match with hard negatives

# Что зашло